# Measurement notebook example

The following tutorials all focus on a specific aspect of arbok. This notebook on the other hand gives as few comments as possible and provides structure of a real world measurement notebook.

## 1. Importing modules

### 1.1 Import of arbok modules

In [1]:
from arbok_driver import (
    ArbokDriver,
    Sample,
    Measurement,
    create_measurement_loop,
    utils
)

2024-12-30 15:38:24,509 - qm - INFO     - Starting session: bc981c6d-12ae-468c-8c11-da7eed121f2d


### 1.2 Import self written sequences and configurations

In [2]:
from example_sequences.square_pulse import SquarePulse
from example_sequences.dummy_readout import DummyReadout

In [3]:
from example_configs.square_pulse_config import square_pulse_conf
from example_configs.dummy_readout_config import dummy_readout_config
from example_configs.dummy_sample import dummy_sample

### 1.3 Generic imports

In [4]:
import numpy as np
import os
%load_ext rich

## 2. Configuring the driver

### 2.1 Instanciating driver, sequence and set relations

In [5]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
dummy_sequence = Measurement(qm_driver, 'dummy_sequence', dummy_sample)
qm_driver.add_sequence(dummy_sequence)

Measrueement init


In [6]:
square_pulse = SquarePulse(dummy_sequence, 'square_pulse', dummy_sample, square_pulse_conf)
readout = DummyReadout(dummy_sequence, 'readout', dummy_sample, dummy_readout_config)

### 2.2 Set sweep parameters and values

In [7]:
dummy_sequence.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    },
    {
        square_pulse.t_square_pulse: np.arange(20, 100, 10, dtype = int)
    }, 
)

Declared 2-dimensional parameter sweep of size 40 [5, 8]


### 2.3 Register the gettables to save

Print gettables of a `ReadSequence` to see all available gettables.

In [8]:
# readout.gettables

In [ ]:
dummy_sequence.register_gettables(
    readout.qubit1.diff(),
    readout.qubit1.read.sensor1_IQ(),
    readout.qubit1.ref.sensor1_IQ(),
)

### 2.4 Compile QUA program and print to file

In [13]:
qm_driver.dummy_sequence

<ArbokDriver: qm_driver>

In [12]:
qua_program = qm_driver.dummy_sequence.get_qua_program()
qm_driver.print_qua_program_to_file(
    file_name = '0_qua_program.py', 
    qua_program = qua_program,
    add_config=False)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 qua_program = qm_driver.dummy_sequence.get_qua_program()                                     │
│   2 qm_driver.print_qua_program_to_file(                                                         │
│   3 │   file_name = '0_qua_program.py',                                                          │
│   4 │   qua_program = qua_program,                                                               │
│                                                                                                  │
│ /home/flatmax/miniconda3/envs/arbok_base/lib/python3.11/site-packages/qcodes/utils/attribute_hel │
│ pers.py:69 in __getattr__                                                                        │
│                                                                                                  │
│    66 │   │   │   except AttributeError:                                                         │
│    67 │   │   │   │   pass                                                                       │
│    68 │   │                                                                                      │
│ ❱  69 │   │   raise AttributeError(                                                              │
│    70 │   │   │   f"'{self.__class__.__name__}' object and its delegates have no attribute '{k   │
│    71 │   │   )                                                                                  │
│    72                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'ArbokDriver' object and its delegates have no attribute 'dummy_sequence'

### 2.5 Connect QM and run program

In [ ]:
IP_ADDRESS = '<YOUR_QM_IP>'
qm_driver.connect_opx(IP_ADDRESS)
qm_driver.run(qua_program)

In [ ]:
sweep_list = [
    {qm_driver.iteration: np.arange(100)},
]

## 3. Set up QCoDeS datamanagement and run measurement 

### 3.1 Define qcodes database path

In [ ]:
db_file_path = os.path.join(os.getcwd(), 'example.db')
initialise_or_create_database_at(db_file_path)

### 3.2 Define the experiment and measurement

In [25]:
MEASUREMENT_NAME = "specific measurement name"

In [ ]:
tutorial_exp = load_or_create_experiment(
    experiment_name="Example Experiment",
    sample_name="dummy_sample"
)
meas = Measurement(exp = tutorial_exp, name = MEASUREMENT_NAME)

### 3.3 Create measurement loop and run the experiment

In [ ]:
@create_measurement_loop(
    sequence = dummy_sequence, measurement=meas, sweep_list=sweep_list)
def run_measurement_loop():
    pass

In [ ]:
run_measurement_loop()